In [ ]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# # Basic packages
import numpy as np
import glob
import os
from pathlib import Path
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import velocereduction as VR

In [ ]:
calibration_runs.keys()

In [ ]:
# Identify the available nights
observations_directory = VR.config.working_directory+'../../observations/'
log_dates = glob.glob(observations_directory+'*')
log_dates = [log_date[-6:] for log_date in log_dates]
log_dates.sort()
if len(log_dates) > 0:
    print('Available dates: ',list(log_dates)[:3],'...',list(log_dates)[-3:])

# Loop through nights and identify the nights without and with LC.
# If there were SimLC exposures, download the last SimLC exposure (following the directory structure)
nights_with_lc = 0
nights_without_lc = 0

nights_with_th = 0
nights_without_th = 0

nights_with_flat = 0
nights_without_flat = 0

simlc_nights_runs = dict()
simth_nights_runs = dict()
fflat_nights_runs = dict()
fflat_nights_runs['ccd1'] = dict()
fflat_nights_runs['ccd2'] = dict()
fflat_nights_runs['ccd3'] = dict()

for log_date in log_dates:

    if (log_date in log_dates[:4]) | (log_date == '240919'):

        # Try to download the log-file
        # command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/*.log '+observations_directory+log_date+'/'
        # os.system(command)

        calibration_runs, science_runs = VR.utils.identify_calibration_and_science_runs(
            log_date,
            observations_directory,
            print_information = False
        )

        print(calibration_runs)

        nr_flats = 0
        if len(calibration_runs['Flat_0.1']) > 0:
            last_fflat = calibration_runs['Flat_0.1'][-1]
            fflat_nights_runs['ccd3'][log_date] = last_fflat
            nr_flats += 1
            command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_3/*3'+last_fflat+'.fits '+observations_directory+log_date+'/ccd_3/'
            print(command)
            os.system(command)
        if len(calibration_runs['Flat_1.0']) > 0:
            last_fflat = calibration_runs['Flat_1.0'][-1]
            fflat_nights_runs['ccd2'][log_date] = last_fflat
            nr_flats += 1
            command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_2/*2'+last_fflat+'.fits '+observations_directory+log_date+'/ccd_2/'
            print(command)
            os.system(command)
        if len(calibration_runs['Flat_60.0']) > 0:
            last_fflat = calibration_runs['Flat_60.0'][-1]
            fflat_nights_runs['ccd1'][log_date] = last_fflat
            nr_flats += 1
            command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_1/*1'+last_fflat+'.fits '+observations_directory+log_date+'/ccd_1/'
            print(command)
            os.system(command)

        if nr_flats != 3:
            nights_without_flat += 1
        else:
            nights_with_flat += 1

        if (len(calibration_runs['SimTh_180.0']) == 0):
            nights_without_th += 1
        else:
            nights_with_th += 1

            last_simth = calibration_runs['SimTh_180.0'][-1]

            # if log_date != '001122':
                # command = 'mkdir '+observations_directory+log_date+'/ccd_1/'
                # os.system(command)
                # command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_1/*1'+last_simth+'.fits '+observations_directory+log_date+'/ccd_1/'
                # print(command)
                # os.system(command)

            simth_nights_runs[log_date] = last_simth

        if (len(calibration_runs['SimTh_180.0']) == 0):
            nights_without_th += 1
        else:
            nights_with_th += 1

            last_simth = calibration_runs['SimTh_180.0'][-1]

            # if log_date != '001122':
                # command = 'mkdir '+observations_directory+log_date+'/ccd_1/'
                # os.system(command)
                # command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_1/*1'+last_simth+'.fits '+observations_directory+log_date+'/ccd_1/'
                # print(command)
                # os.system(command)

            simth_nights_runs[log_date] = last_simth


        if (len(calibration_runs['SimLC']) == 0) | (log_date in ['240118','240621','250624','250629']):
            nights_without_lc += 1
        else:
            nights_with_lc += 1

            last_simlc = calibration_runs['SimLC'][-1]

            # # Create directories
            # command = 'mkdir '+observations_directory+log_date+'/ccd_2/'
            # os.system(command)
            # command = 'mkdir '+observations_directory+log_date+'/ccd_3/'
            # os.system(command)

            # command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_2/*2'+last_simlc+'.fits '+observations_directory+log_date+'/ccd_2/'
            # print(command)
            # os.system(command)
            # command = 'rsync -azu -P galah@galahobs.datacentral.org.au:/galah/obs/veloce_halo/'+log_date+'/ccd_3/*3'+last_simlc+'.fits '+observations_directory+log_date+'/ccd_3/'
            # print(command)
            # os.system(command)

            simlc_nights_runs[log_date] = last_simlc

    # print('Nights with LC:',nights_with_lc)
    # print('Nights without LC:',nights_without_lc)

    # print('Nights with ThXe:',nights_with_th)
    # print('Nights without ThXe:',nights_without_th)


In [ ]:
print(simth_nights_runs)
print(simlc_nights_runs)
print(fflat_nights_runs)

In [ ]:
date = '001122'

# SimTh
# ccd = 1; run = simth_nights_runs[date]
# full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
# ref_ccd1, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)
# ccd = 2; run = simlc_nights_runs[date]
# full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
# ref_ccd2, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)
# ccd = 3; run = simlc_nights_runs[date]
# full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
# ref_ccd3, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

# Flats
ccd = 1; run = fflat_nights_runs['ccd1'][date]
full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
ref_ccd1, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

ccd = 2; run = fflat_nights_runs['ccd2'][date]
full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
ref_ccd2, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

ccd = 3; run = fflat_nights_runs['ccd3'][date]
full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
ref_ccd3, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)


In [ ]:
position = '2k_2k'
# position = '1k_1k'

use_flat_or_simth = 'flat'

for date in list(simth_nights_runs.keys()):

    if date == '240919':
    # if date in simlc_nights_runs.keys():

        if use_flat_or_simth == 'flat':
            print('Using Flat not SimTh for shift estimate!')
            run = fflat_nights_runs['ccd1'][date]
        else:
            run = simth_nights_runs[date]
        ccd = 1
        full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
        trimmed_image_ccd1, os_median_ccd1, os_rms_ccd1, readout_mode_ccd1 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

        print(date, run, ccd, np.shape(trimmed_image_ccd1))

        # SimLC for CCD2
        ccd = 2
        if use_flat_or_simth == 'flat':
            run = fflat_nights_runs['ccd2'][date]
        else:
            run = simlc_nights_runs[date]
        full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
        trimmed_image_ccd2, os_median_ccd2, os_rms_ccd2, readout_mode_ccd2 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

        print(date, run, ccd, np.shape(trimmed_image_ccd2))

        # SimLC for CCD3
        ccd = 3
        if use_flat_or_simth == 'flat':
            run = fflat_nights_runs['ccd3'][date]
        else:
            run = simlc_nights_runs[date]

        full_image, metadata = VR.extraction.read_veloce_fits_image_and_metadata(observations_directory+date+'/ccd_'+str(ccd)+'/'+date[-2:]+VR.utils.match_month_to_date(date)+str(ccd)+run+'.fits')
        trimmed_image_ccd3, os_median_ccd3, os_rms_ccd3, readout_mode_ccd3 = VR.extraction.substract_overscan(full_image, metadata, debug_overscan=False)

        print(date, run, ccd, np.shape(trimmed_image_ccd3))

        f, (ax_ccd1, ax_ccd2, ax_ccd3) = plt.subplots(1,3,figsize=(8,4))

        dx1, dy1, de1 = VR.utils.phase_correlation_shift(ref_ccd1, trimmed_image_ccd1)
        dx2, dy2, de2 = VR.utils.phase_correlation_shift(ref_ccd2, trimmed_image_ccd2)
        dx3, dy3, de3 = VR.utils.phase_correlation_shift(ref_ccd3, trimmed_image_ccd3)

        ax_ccd1.text(0.05,0.05,f'dx ${np.round(dx1,2):+.2f}$\ndy ${np.round(dy1,2):+.2f}$',transform=ax_ccd1.transAxes,ha='left',va='bottom')
        ax_ccd2.text(0.05,0.05,f'dx ${np.round(dx2,2):+.2f}$\ndy ${np.round(dy2,2):+.2f}$',transform=ax_ccd2.transAxes,ha='left',va='bottom')
        ax_ccd3.text(0.05,0.05,f'dx ${np.round(dx3,2):+.2f}$\ndy ${np.round(dy3,2):+.2f}$',transform=ax_ccd3.transAxes,ha='left',va='bottom')

        ax_ccd1.set_title(date + ' CCD1 SimTh '+readout_mode_ccd2)
        ax_ccd2.set_title(date + ' CCD2 SimLC '+readout_mode_ccd2)
        ax_ccd3.set_title(date + ' CCD3 SimLC '+readout_mode_ccd3)

        trimmed_image_ccd1 /= np.percentile(trimmed_image_ccd1,q=95)
        trimmed_image_ccd2 /= np.percentile(trimmed_image_ccd2,q=95)
        trimmed_image_ccd3 /= np.percentile(trimmed_image_ccd3,q=95)

        h_ccd1 = ax_ccd1.imshow(trimmed_image_ccd1, cmap = 'Reds', norm=LogNorm(vmin=1, vmax = 10**3), origin='lower')
        cbar_ccd1 = plt.colorbar(h_ccd1, ax=ax_ccd1)
        h_ccd2 = ax_ccd2.imshow(trimmed_image_ccd2, cmap = 'Reds', norm=LogNorm(vmin=10, vmax = 10**4), origin='lower')
        cbar_ccd2 = plt.colorbar(h_ccd2, ax=ax_ccd2)
        h_ccd3 = ax_ccd3.imshow(trimmed_image_ccd3, cmap = 'Reds', norm=LogNorm(vmin=10, vmax = 10**4), origin='lower')
        cbar_ccd3 = plt.colorbar(h_ccd3, ax=ax_ccd3)

        if position == '2k_2k':
            ax_ccd1.set_xlim(1992.5,2055.5)
            ax_ccd1.set_ylim(2020.5,2203.5)
            ax_ccd2.set_xlim(2042.5,2058.5)
            ax_ccd2.set_ylim(2004.5,2043.5)
            ax_ccd3.set_xlim(2008.5,2020.5)
            ax_ccd3.set_ylim(2037.5,2090.5)
        if position == '1k_1k':
            ax_ccd1.set_xlim(1020.5,1040.5)
            ax_ccd1.set_ylim(1000.5,1050.5)
            ax_ccd2.set_xlim(1020.5,1040.5)
            ax_ccd2.set_ylim(1000.5,1050.5)
            ax_ccd3.set_xlim(959.5,978.5)
            ax_ccd3.set_ylim(1003.5,1050.5)

        plt.tight_layout()
        # plt.savefig('shift_monitoring/'+position+'/shift_monitoring'+position+'_'+date+'.png',dpi=200,bbox_inches='tight')
        plt.show()
        plt.close()

        # f2, ax = plt.subplots(figsize=(16,16))
        # ax.imshow(trimmed_image_ccd1, cmap = 'Reds', norm=LogNorm(vmin=1, vmax = 10**3), origin='lower')
        # # ax.set_xlim(1900,2150)
        # # ax.set_ylim(1900,2150)
        # plt.show(f2)
        # plt.close(f2)

    # command = """
    # magick $(ls shift_monitoring/'+position+'/shift_monitoring*.png | sort -V) \
    #   -background white -alpha remove -alpha off \
    #   -set dispose background \
    #   -delay 100 -loop 0 \
    #   shift_monitoring/shift_monitoring_'+position+'.gif
    # """
    # os.system(command)

In [ ]:
# # Identify all previous wavelength solutions
# wavelength_coefficient_directory = str(Path(VR.config.working_directory).resolve())
# wavelength_coefficient_directories = np.concatenate(([glob.glob(wavelength_coefficient_directory+'/'+m+'*') for m in ['0','1','2']]))
# dates = [directory[-6:] for directory in wavelength_coefficient_directories]
# dates.sort()

# lc_orders = glob.glob(wavelength_coefficient_directories[0]+'/*lc.txt')
# lc_orders = np.array([lc_order.split('/')[-1][24:-7] for lc_order in lc_orders])

# lc_monitor_table = {}

# for lc_order in lc_orders:
#     rows = []

#     for date in dates:
#         fname = (
#             Path(wavelength_coefficient_directory)
#             / date
#             / f"wavelength_coefficients_{lc_order}_lc.txt"
#         )

#         coeffs = np.loadtxt(fname)
#         rows.append((date, *coeffs))

#     lc_monitor_table[lc_order] = Table(
#         rows=rows,
#         names=[
#             "date",
#             "coeff_0", "coeff_1", "coeff_2",
#             "coeff_3", "coeff_4", "coeff_5",
#         ]
#     )


In [ ]:
# for order in list(lc_monitor_table.keys()):
#     if order in ['ccd_2_order_120','ccd_3_order_93']:
#         f, gs = plt.subplots(3,2,figsize=(10,4), sharex=True)
#         gs = gs.flatten()
#         for index, key in enumerate((lc_monitor_table[order].keys())[1:]):
#             ax = gs[index]

#             mean = np.mean(lc_monitor_table[order][key])
#             std  = np.std(lc_monitor_table[order][key])
#             change_percent = np.round(100*std/mean,4)

#             ax.scatter(
#                 lc_monitor_table[order]['date'],
#                 lc_monitor_table[order][key],
#                 label = 'Change: '+str(change_percent)+'%'
#             )
#             ax.legend()
#             ax.tick_params(axis="x", labelrotation=45)
#         plt.tight_layout(w_pad=0,h_pad=0)
#         plt.show()